# Emission files for Sven's test case

Produce point source emission and corresponding temporal and vertical profiles for Sven Werchner's simple test case with H2O2 chemistry

- produce point-source emission file for OEM
- produce vertical profile
- produce temporal profile

## Create Point source emission file
The file is created for the ICON grid provided by Sven with two point sources releasing H2O2

In [4]:
# imports
from pathlib import Path
from emiproc.inventories import Inventory
from emiproc.grids import WGS84, ICONGrid, WGS84_PROJECTED

from emiproc.inventories.utils import group_categories
from emiproc.regrid import remap_inventory
from emiproc.utilities import SEC_PER_YR
from emiproc.exports.icon import export_icon_oem

from shapely.geometry import Point 
import geopandas as gpd
import yaml

In [6]:
# create inventory object from a GeoDataFrame (gdf) with the two point sources
inv=Inventory.from_gdf(gdfs={'point_sources': gpd.GeoDataFrame(
    {
    "H2O2": [4*SEC_PER_YR,4*SEC_PER_YR],
    },
    geometry=[Point(8.4,49.0), Point(9.2,48.8)],
              crs="EPSG:4326",
              )}
)

# Convert to a planar crs for the remapping to work
inv.to_crs(WGS84_PROJECTED)

In [7]:
# Load  icon grid and remap the inventory to this grid
grid_file = Path(
    r"/store/empa/em05/dbrunner/icon-art/werchner/grid4OEM/BASE_KA_DOM01.nc"
)
icon_grid = ICONGrid(grid_file)

remapped_point = remap_inventory(inv, icon_grid, grid_file.parent / f"remapped_point_{grid_file.stem}")

In [8]:
# export to ICON-compatible file
export_icon_oem(remapped_point, grid_file, grid_file.with_stem(f"{grid_file.stem}_with_h2o2_emissions"))

ImportError: The cartopy package is required to compute the country mask.

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np

import matplotlib.pyplot as plt

import emiproc
from emiproc.profiles.temporal_profiles import (
    TemporalProfile,
    DailyProfile,
    WeeklyProfile,
    MounthsProfile,
    create_scaling_factors_time_serie,
    from_csv,
    from_yaml,
    to_yaml,
    read_temporal_profiles,
)
from emiproc.profiles import vertical_profiles 
from emiproc.tests_utils.temporal_profiles import copernicus_profiles_dir

## Vertical profiles

In [ ]:
# Load from csv

profiles, cat_sub = vertical_profiles.from_csv(
    copernicus_profiles_dir / "vertical.csv"
)   

In [ ]:
# Plot the profiles
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
for name, profile in zip(cat_sub, profiles.ratios):
    ax.plot(profile, label=name)
ax.legend()

## Time profiles

In [ ]:
read_temporal_profiles(copernicus_profiles_dir)

In [ ]:
# Profiles from a csv profiles file 


profiles = ['hour_in_day', 'day_in_week', 'month_in_year']
profiles = {p: from_csv(copernicus_profiles_dir / f"timeprofiles-{p}.csv") for p in profiles}

# Plot the profiles on 3 subplots and put the legend on a 4rth 
fig, axs = plt.subplots(2, 2, figsize=(10, 10))
axs = axs.flatten()
for ax, (name, profile) in zip(axs, profiles.items()):
    for cat_name, cat_profile in profile.items():
        ax.plot(cat_profile.ratios, label=cat_name)
    ax.set_title(name)

for cat_name, cat_profile in profile.items():
    axs[-1].plot([], label=cat_name)
axs[-1].legend()




In [ ]:
# Profiles from a yaml profiles file
yaml_dir = emiproc.FILES_DIR / "profiles" / "yamls"
yaml_profiles = {}
for yml_file in yaml_dir.glob("*.yaml"):
    yaml_profiles[yml_file.stem] = from_yaml(yml_file)

# Now with the yaml, each catoegory can have different types of profiles
yaml_profiles['heat'], yaml_profiles['heavy']


In [ ]:
# We can create an hourly time series of each categories to plot
tss = {}
for categorie in yaml_profiles.keys():
    tss[categorie] = create_scaling_factors_time_serie(
        start_time="2020-01-01",
        end_time="2022-04-28",
        profiles=yaml_profiles[categorie],
    )
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
# Create better colors 
colors = iter(plt.cm.tab10(np.linspace(0, 1, len(tss))))
for name, ts in tss.items():
    ax.plot(ts, label=name, color=next(colors))
ax.legend()

In [ ]:
# We can also save time profiles to yaml
for categorie in yaml_profiles.keys():
    yaml_file = emiproc.FILES_DIR / 'outputs' / f'test_{categorie}.yaml'
    to_yaml(yaml_profiles[categorie], yaml_file)


### TNO profiles

In [ ]:
from pathlib import Path
import xarray as xr

from emiproc.inventories.tno import TNO_Inventory

tno_nc_file = Path(r"C:\Users\coli\Documents\emiproc\files\TNO_6x6_GHGco_v4_0\TNO_GHGco_v4_0_year2018.nc")


inv_tno = TNO_Inventory(tno_nc_file)

ICON Profiles

In [ ]:
from pathlib import Path
import xarray as xr

nc_file = Path(r"C:\Users\coli\Documents\ZH-CH-emission\icon_europe_DOM01_with_tno_emissions_test_for_nikolai_bug\hourofyear.nc")

ds = xr.open_dataset(nc_file)
ds


In [ ]:
cat = "GNFR_R-CO2"
country = 0
ds[cat].sel(country=country).plot()